# Molecular Generator Evaluation using TUPOR, SESY and ASER Metrics

🔹 **Objective**  
This notebook evaluates molecular generators by computing four key metrics:  
   - **TUPOR**: scaffold recall metrics  
   - **SESY**: scaffold hopping potencial  
   - **ASER**: chemical space exploration

🔹 **Workflow**  
1️⃣ **Compute Metrics**: The script calculates TUPOR, SESY and ASER for different molecular generators.  
2️⃣ **Merge Data**: Results from multiple generators are combined into a single Pandas DataFrame.  
3️⃣ **Normalize Values**: The computed metrics are normalized using Min-Max scaling for comparison.  
4️⃣ **Save Outputs**: Processed data is stored in CSV files for further analysis.  

🔹 **Data Structure**  
- The calculations are performed for different **scaffold types** (`csk`, `murcko`) and **cluster types** (`dis`, `sim`).  
- Results are computed for multiple **generators** (`Molpher`, `REINVENT`, `DrugEx`, `GB_GA`, etc.).  
- The analysis is conducted for a specific **biological target receptor**, such as the **Glucocorticoid receptor**.

This notebook allows us to compare the performance of various molecular generators in terms of structural diversity, similarity to known bioactive compounds, and synthetic feasibility.

# Loading required libraries

In [11]:
from src import metrics # Importing custom metric functions
import importlib as imp
imp.reload(metrics)

<module 'src.metrics' from '/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/src/metrics.py'>

# Function to calculate metrics

In [14]:
def calculate_metrics(type_cluster, type_scaffold,generator, receptor, ncpus = 1):
    """
    Function to calculate molecular generation metrics.
    
    Parameters:
    - scaffold_type: Type of scaffold (e.g., 'csk' or 'murcko')
    - type_cluster: Cluster type  (e.g., 'dis' or 'sim')
    - generator: Name of the molecular generator
    - receptor: Target receptor for drug design

    Returns:
    - Computed metrics
    """
    mt = metrics.Metrics(type_cluster, type_scaffold, generator, receptor, ncpus)     
    result = mt.calculate_metrics()
    display(result)
    return result

# Define parameters for metric calculations

In [15]:
type_cluster = 'dis' #options: 'dis'|'sim' 
type_scaffold = 'csk' #options: 'csk'|'murcko'
generator = 'GB_GA_mut_r_0.5' #options: 'Molpher'|'DrugEx'|'REINVENT'|'addcarbon'
receptor = 'Leukocyte_elastase' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

calculate_metrics(type_cluster,type_scaffold,generator,receptor, ncpus = 10)

NUMBER:  0


[09:54:37] Explicit valence for atom # 8 C, 5, is greater than permitted
[09:54:38] Explicit valence for atom # 18 C, 5, is greater than permitted
[09:54:39] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:54:40] Explicit valence for atom # 2 C, 5, is greater than permitted
[09:54:41] Explicit valence for atom # 9 C greater than permitted
[09:54:42] Explicit valence for atom # 2 C, 5, is greater than permitted
[09:54:43] Explicit valence for atom # 2 C, 5, is greater than permitted
[09:54:43] Explicit valence for atom # 13 C greater than permitted
[09:54:48] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:54:48] Explicit valence for atom # 8 C, 5, is greater than permitted
[09:54:49] Explicit valence for atom # 13 C greater than permitted
[09:54:52] Explicit valence for atom # 16 C, 5, is greater than permitted
[09:54:53] Explicit valence for atom # 16 C, 5, is greater than permitted
[09:54:53] Explicit valence for atom # 1 C, 5, is greater than perm

NUMBER:  1


[09:57:19] Explicit valence for atom # 2 C, 5, is greater than permitted
[09:57:21] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:57:21] Explicit valence for atom # 1 C greater than permitted
[09:57:22] Explicit valence for atom # 1 C greater than permitted
[09:57:23] Explicit valence for atom # 6 C, 5, is greater than permitted
[09:57:30] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:57:31] Explicit valence for atom # 1 C greater than permitted
[09:57:31] Explicit valence for atom # 16 C, 5, is greater than permitted
[09:57:34] Explicit valence for atom # 7 C greater than permitted
[09:57:35] Explicit valence for atom # 14 C, 5, is greater than permitted
[09:57:39] Explicit valence for atom # 1 C greater than permitted
[09:57:47] Explicit valence for atom # 1 C greater than permitted
[09:57:50] Explicit valence for atom # 12 C, 5, is greater than permitted
[09:57:51] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:57:52] Explic

NUMBER:  2


...

[10:02:34] Explicit valence for atom # 2 C greater than permitted
[10:02:37] Explicit valence for atom # 12 C greater than permitted
[10:02:37] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:02:37] Explicit valence for atom # 7 C greater than permitted
[10:02:38] Explicit valence for atom # 19 C, 5, is greater than permitted
[10:02:38] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:02:39] Explicit valence for atom # 8 C greater than permitted
[10:02:39] Explicit valence for atom # 9 C greater than permitted
[10:02:40] Explicit valence for atom # 6 C greater than permitted
[10:02:40] Explicit valence for atom # 19 C greater than permitted
[10:02:41] Explicit valence for atom # 1 C greater than permitted
[10:02:41] Explicit valence for atom # 6 C greater than permitted
[10:02:41] Explicit valence for atom # 20 C greater than permitted
[10:02:42] Explicit valence for atom # 9 C greater than permitted
[10:02:42] Explicit valence for atom # 21 C, 5, is 

NUMBER:  4


[10:05:14] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:05:14] Explicit valence for atom # 3 C greater than permitted
[10:05:16] Explicit valence for atom # 1 C greater than permitted
[10:05:16] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:05:16] Explicit valence for atom # 6 C greater than permitted
[10:05:16] Explicit valence for atom # 21 C greater than permitted
[10:05:16] Explicit valence for atom # 4 C, 5, is greater than permitted
[10:05:16] Explicit valence for atom # 8 C, 5, is greater than permitted
[10:05:17] Explicit valence for atom # 6 C, 5, is greater than permitted
[10:05:17] Explicit valence for atom # 7 C, 5, is greater than permitted
[10:05:17] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:05:17] Explicit valence for atom # 11 C, 5, is greater than permitted
[10:05:17] Explicit valence for atom # 26 C greater than permitted
[10:05:18] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:05:18]

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,GB_GA_mut_r_0.5_0,dis,csk,19321.0,856784.0,29/42,0.690476,0.022551,0.064611,51998.0
1,GB_GA_mut_r_0.5_1,dis,csk,18763.0,845730.0,19/41,0.463415,0.022186,0.005752,4837.0
2,GB_GA_mut_r_0.5_2,dis,csk,18755.0,859652.0,31/45,0.688889,0.021817,0.000870,747.0
3,GB_GA_mut_r_0.5_3,dis,csk,20028.0,884523.0,26/47,0.553191,0.022643,0.514384,300442.0
4,GB_GA_mut_r_0.5_4,dis,csk,17081.0,852028.0,25/41,0.609756,0.020047,0.014424,12115.0
5,GB_GA_mut_r_0.5_mean,dis,csk,18789.6,859743.4,-,0.601145,0.021849,0.120008,74027.8


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,GB_GA_mut_r_0.5_0,dis,csk,19321.0,856784.0,29/42,0.690476,0.022551,0.064611,51998.0
1,GB_GA_mut_r_0.5_1,dis,csk,18763.0,845730.0,19/41,0.463415,0.022186,0.005752,4837.0
2,GB_GA_mut_r_0.5_2,dis,csk,18755.0,859652.0,31/45,0.688889,0.021817,0.000870,747.0
3,GB_GA_mut_r_0.5_3,dis,csk,20028.0,884523.0,26/47,0.553191,0.022643,0.514384,300442.0
4,GB_GA_mut_r_0.5_4,dis,csk,17081.0,852028.0,25/41,0.609756,0.020047,0.014424,12115.0
5,GB_GA_mut_r_0.5_mean,dis,csk,18789.6,859743.4,-,0.601145,0.021849,0.120008,74027.8


# Execute metric calculation function

In [4]:
for receptor in ['Leukocyte_elastase']:
    for type_scaffold in ['csk']:
        for type_cluster in ['sim']:
            for subset in ['_300k']:
                save_options =True
                ncpus = 7
                
                # Define generator names with different epsilon values
                generators_name_list = [
                    f"Molpher{subset}",
                    f"REINVENT{subset}",
                    f"DrugEx_GT_epsilon_0.1{subset}",
                    f"DrugEx_GT_epsilon_0.6{subset}",
                    f"DrugEx_RNN_epsilon_0.1{subset}",
                    f"DrugEx_RNN_epsilon_0.6{subset}",
                    f"GB_GA_mut_r_0.01{subset}",
                    f"GB_GA_mut_r_0.5{subset}",
                    f"addcarbon{subset}"
                ]
                for generator in generators_name_list:
                    calculate_metrics(type_cluster,type_scaffold,generator,receptor,save_options, ncpus = ncpus)

NUMBER:  0


[15:19:55] Explicit valence for atom # 14 C, 5, is greater than permitted


NUMBER:  1


[15:20:27] Explicit valence for atom # 2 C, 5, is greater than permitted
[15:20:27] Explicit valence for atom # 24 C, 5, is greater than permitted
[15:20:27] Explicit valence for atom # 24 C, 5, is greater than permitted
[15:20:32] Explicit valence for atom # 11 C, 5, is greater than permitted
[15:20:32] Explicit valence for atom # 11 C, 5, is greater than permitted
[15:20:32] Explicit valence for atom # 11 C, 5, is greater than permitted
[15:20:53] Explicit valence for atom # 7 C, 5, is greater than permitted


NUMBER:  2


...

NUMBER:  4


[16:07:50] Explicit valence for atom # 10 C greater than permitted
[16:07:51] Explicit valence for atom # 1 C, 5, is greater than permitted
[16:07:51] Explicit valence for atom # 1 C greater than permitted
[16:07:52] Explicit valence for atom # 6 C greater than permitted
[16:07:52] Explicit valence for atom # 1 C, 5, is greater than permitted
[16:07:53] Explicit valence for atom # 12 C, 5, is greater than permitted
[16:07:53] Explicit valence for atom # 2 C greater than permitted
[16:07:53] Explicit valence for atom # 18 C, 5, is greater than permitted
[16:07:54] Explicit valence for atom # 9 C, 5, is greater than permitted
[16:07:54] Explicit valence for atom # 6 C greater than permitted
[16:07:54] Explicit valence for atom # 10 C, 5, is greater than permitted
[16:07:55] Explicit valence for atom # 3 C, 5, is greater than permitted
[16:07:55] Explicit valence for atom # 1 C, 5, is greater than permitted
[16:07:55] Explicit valence for atom # 1 C greater than permitted
[16:07:55] Expli

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,GB_GA_mut_r_0.5_300k_0,sim,csk,14627.0,272184.0,38/46,0.826087,0.053739,0.375111,74248.0
1,GB_GA_mut_r_0.5_300k_1,sim,csk,8292.0,254131.0,29/44,0.659091,0.032629,0.017941,4479.0
2,GB_GA_mut_r_0.5_300k_2,sim,csk,8157.0,255324.0,31/42,0.738095,0.031948,0.036996,9109.0
3,GB_GA_mut_r_0.5_300k_3,sim,csk,8783.0,257159.0,24/42,0.571429,0.034154,0.092481,21769.0
4,GB_GA_mut_r_0.5_300k_4,sim,csk,8811.0,257451.0,21/42,0.500000,0.034224,0.003348,859.0
5,GB_GA_mut_r_0.5_300k_mean,sim,csk,9734.0,259249.8,-,0.658940,0.037339,0.105175,22092.8


NUMBER:  0
NUMBER:  1
NUMBER:  2
NUMBER:  3
NUMBER:  4


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,addcarbon_300k_0,sim,csk,953.0,300000.0,21/46,0.456522,0.003177,0.030220,8800.0
1,addcarbon_300k_1,sim,csk,941.0,300000.0,16/44,0.363636,0.003137,0.010404,3089.0
2,addcarbon_300k_2,sim,csk,965.0,300000.0,12/42,0.285714,0.003217,0.009985,2966.0
3,addcarbon_300k_3,sim,csk,910.0,300000.0,15/42,0.357143,0.003033,0.010696,3175.0
4,addcarbon_300k_4,sim,csk,935.0,300000.0,12/42,0.285714,0.003117,0.003365,1006.0
5,addcarbon_300k_mean,sim,csk,940.8,300000.0,-,0.349746,0.003136,0.012934,3807.2


# Run function to combine mean metric values into a single pandas DataFrame and to combine mean values into single DataFrame with normalization MinMax

In [ ]:
from src import metrics_connection # Importing custom metric functions
imp.reload(metrics_connection)

In [31]:
for type_scaffold in ['csk', 'murcko']:
    for type_cluster in ['dis', 'sim']:  # Different cluster types
        for subset in ['_62.5k', '_125k', '_250k']:
        
            # Define generator names with different epsilon values
            generators_name_list = [
                f"Molpher{subset}",
                f"REINVENT{subset}",
                f"DrugEx_GT_epsilon_0.1{subset}",
                f"DrugEx_GT_epsilon_0.6{subset}",
                f"DrugEx_RNN_epsilon_0.1{subset}",
                f"DrugEx_RNN_epsilon_0.6{subset}",
                f"GB_GA_mut_r_0.01{subset}",
                f"GB_GA_mut_r_0.5{subset}",
                f"addcarbon{subset}"
            ]

            # Connect and process mean values
            df = metrics_connection.connect_mean_value(type_cluster, type_scaffold, generators_name_list, receptor, subset)
            df1 = metrics_connection.connect_mean_value_normalized(type_cluster, type_scaffold, generators_name_list, receptor, subset):

,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_mean,dis,csk,202012.0,1601325.6,0.732956,0.126167,0.005663,0.000158,9027.0
1,REINVENT_mean,dis,csk,83979.0,678050.4,0.592146,0.123426,0.005181,0.000307,3390.2
2,DrugEx_GT_epsilon_0.1_mean,dis,csk,170925.8,873782.0,0.726759,0.195085,0.020413,0.000194,17245.6
3,DrugEx_GT_epsilon_0.6_mean,dis,csk,231874.4,898840.6,0.779855,0.257924,0.019590,0.000146,17236.0
4,DrugEx_RNN_epsilon_0.1_mean,dis,csk,66202.2,873267.6,0.675397,0.075972,0.049605,0.000453,40088.6
5,DrugEx_RNN_epsilon_0.6_mean,dis,csk,88390.6,881881.0,0.805893,0.100221,0.035630,0.000400,29384.6
6,GB_GA_mut_r_0.01_mean,dis,csk,56614.6,896237.6,0.364403,0.061576,0.014584,0.000312,12469.4
7,GB_GA_mut_r_0.5_mean,dis,csk,18789.6,859743.4,0.601145,0.021849,0.120008,0.001388,74027.8
8,addcarbon_mean,dis,csk,930.0,1000000.0,0.137921,0.000930,0.004051,0.006610,4028.0


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_10k_mean,dis,csk,2396.0,10109.0,0.155461,0.236962,0.005214,0.002877,52.2
1,REINVENT_10k_mean,dis,csk,4240.2,9869.0,0.240552,0.429577,0.005152,0.002382,50.6
2,DrugEx_GT_epsilon_0.1_10k_mean,dis,csk,4971.6,9989.4,0.308595,0.497691,0.020693,0.002778,198.8
3,DrugEx_GT_epsilon_0.6_10k_mean,dis,csk,5409.6,9951.4,0.406764,0.543623,0.019799,0.003250,192.4
4,DrugEx_RNN_epsilon_0.1_10k_mean,dis,csk,2443.8,9855.6,0.262111,0.247675,0.047766,0.004766,430.6
5,DrugEx_RNN_epsilon_0.6_10k_mean,dis,csk,3006.8,9819.0,0.363309,0.306063,0.035684,0.005383,329.8
6,GB_GA_mut_r_0.01_10k_mean,dis,csk,2366.8,8955.0,0.125668,0.258043,0.014036,0.002593,119.6
7,GB_GA_mut_r_0.5_10k_mean,dis,csk,996.6,8594.2,0.212776,0.115922,0.118979,0.009259,731.2
8,addcarbon_10k_mean,dis,csk,728.8,10000.0,0.115399,0.072880,0.004405,0.007149,43.8


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_100k_mean,dis,csk,19439.0,100090.2,0.494166,0.194214,0.005937,0.001115,590.0
1,REINVENT_100k_mean,dis,csk,22554.0,98728.0,0.419493,0.228391,0.005053,0.000792,496.0
2,DrugEx_GT_epsilon_0.1_100k_mean,dis,csk,31608.4,99859.8,0.503196,0.316560,0.020194,0.000714,1942.2
3,DrugEx_GT_epsilon_0.6_100k_mean,dis,csk,37781.8,99544.2,0.642068,0.379568,0.019414,0.000736,1888.2
4,DrugEx_RNN_epsilon_0.1_100k_mean,dis,csk,13465.4,98572.4,0.502378,0.136462,0.049541,0.001652,4440.4
5,DrugEx_RNN_epsilon_0.6_100k_mean,dis,csk,17350.0,98276.0,0.639303,0.176408,0.035621,0.001638,3294.6
6,GB_GA_mut_r_0.01_100k_mean,dis,csk,12501.6,89583.6,0.254407,0.136061,0.014540,0.000990,1242.8
7,GB_GA_mut_r_0.5_100k_mean,dis,csk,4473.6,85964.6,0.415459,0.052015,0.121313,0.004031,7457.0
8,addcarbon_100k_mean,dis,csk,922.6,100000.0,0.137921,0.009226,0.004008,0.006662,398.6


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_500k_mean,dis,csk,77545.0,500057.4,0.652289,0.155072,0.005777,0.000366,2869.4
1,REINVENT_500k_mean,dis,csk,67556.2,493615.6,0.559795,0.136815,0.005152,0.000355,2528.0
2,DrugEx_GT_epsilon_0.1_500k_mean,dis,csk,110691.0,499303.6,0.689213,0.221721,0.020400,0.000281,9795.2
3,DrugEx_GT_epsilon_0.6_500k_mean,dis,csk,142793.4,497711.0,0.743566,0.286916,0.019507,0.000226,9485.2
4,DrugEx_RNN_epsilon_0.1_500k_mean,dis,csk,43760.8,492809.4,0.656319,0.088708,0.049549,0.000665,22192.8
5,DrugEx_RNN_epsilon_0.6_500k_mean,dis,csk,57322.8,491287.0,0.767450,0.116581,0.035681,0.000593,16493.6
6,GB_GA_mut_r_0.01_500k_mean,dis,csk,36395.6,448135.4,0.341692,0.079175,0.014550,0.000458,6218.6
7,GB_GA_mut_r_0.5_500k_mean,dis,csk,12250.4,429775.4,0.550076,0.028494,0.120252,0.001946,37072.4
8,addcarbon_500k_mean,dis,csk,930.0,500000.0,0.137921,0.001860,0.004028,0.006610,2002.6


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_mean,sim,csk,211411.2,1661166.8,0.835874,0.127299,0.012845,0.000172,20763.6
1,REINVENT_mean,sim,csk,79372.8,723954.8,0.645464,0.109586,0.010646,0.000357,7640.8
2,DrugEx_GT_epsilon_0.1_mean,sim,csk,173618.0,888053.6,0.799473,0.194621,0.021014,0.000207,18300.6
3,DrugEx_GT_epsilon_0.6_mean,sim,csk,238449.0,908235.8,0.896518,0.262326,0.027222,0.000167,23819.2
4,DrugEx_RNN_epsilon_0.1_mean,sim,csk,72688.4,942638.6,0.658761,0.076307,0.066587,0.000400,53886.2
5,DrugEx_RNN_epsilon_0.6_mean,sim,csk,102034.6,898509.8,0.859467,0.113405,0.048223,0.000370,40238.4
6,GB_GA_mut_r_0.01_mean,sim,csk,72421.0,913169.2,0.474101,0.076706,0.017485,0.000381,15686.0
7,GB_GA_mut_r_0.5_mean,sim,csk,20436.2,864169.4,0.724977,0.023512,0.105139,0.001587,73608.4
8,addcarbon_mean,sim,csk,940.8,1000000.0,0.349746,0.000941,0.012993,0.016436,12747.4


...

,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_500k_mean,dis,murcko,164411.2,500060.4,0.182822,0.328783,0.002502,0.000099,1246.4
1,REINVENT_500k_mean,dis,murcko,287052.4,493658.2,0.269618,0.581152,0.000737,0.000087,364.2
2,DrugEx_GT_epsilon_0.1_500k_mean,dis,murcko,311584.4,499374.2,0.265356,0.623952,0.007417,0.000077,3639.4
3,DrugEx_GT_epsilon_0.6_500k_mean,dis,murcko,309659.2,497770.6,0.415078,0.622108,0.006060,0.000120,2987.2
4,DrugEx_RNN_epsilon_0.1_500k_mean,dis,murcko,222355.4,493506.4,0.111378,0.450317,0.015149,0.000041,7277.4
5,DrugEx_RNN_epsilon_0.6_500k_mean,dis,murcko,249408.0,492216.6,0.305735,0.506531,0.009523,0.000113,4611.0
6,GB_GA_mut_r_0.01_500k_mean,dis,murcko,118929.8,448135.4,0.056883,0.260800,0.007070,0.000046,2986.2
7,GB_GA_mut_r_0.5_500k_mean,dis,murcko,74103.6,429878.0,0.117586,0.172346,0.044205,0.000134,16686.4
8,addcarbon_500k_mean,dis,murcko,5142.8,500000.0,0.029286,0.010286,0.000319,0.000606,159.2


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_mean,sim,murcko,505888.8,1661210.0,0.465611,0.304545,0.004591,0.000083,7462.8
1,REINVENT_mean,sim,murcko,412546.8,724047.0,0.292322,0.569795,0.001216,0.000063,912.8
2,DrugEx_GT_epsilon_0.1_mean,sim,murcko,511699.6,888095.0,0.427641,0.574657,0.005648,0.000077,5060.8
3,DrugEx_GT_epsilon_0.6_mean,sim,murcko,558278.0,908399.4,0.645077,0.614167,0.008563,0.000104,7637.8
4,DrugEx_RNN_epsilon_0.1_mean,sim,murcko,395997.2,945613.2,0.104826,0.418035,0.019666,0.000026,16708.6
5,DrugEx_RNN_epsilon_0.6_mean,sim,murcko,452293.2,901145.0,0.562998,0.501581,0.011972,0.000113,10491.8
6,GB_GA_mut_r_0.01_mean,sim,murcko,239421.0,913169.2,0.067379,0.256611,0.003464,0.000028,3127.6
7,GB_GA_mut_r_0.5_mean,sim,murcko,136899.4,864302.6,0.307261,0.157913,0.037890,0.000205,29577.4
8,addcarbon_mean,sim,murcko,5150.8,1000000.0,0.118272,0.005151,0.001690,0.001652,1685.8


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_10k_mean,sim,murcko,3700.6,10102.6,0.053786,0.366380,0.007329,0.001547,72.8
1,REINVENT_10k_mean,sim,murcko,9214.4,9895.2,0.031186,0.931202,0.001322,0.000368,13.0
2,DrugEx_GT_epsilon_0.1_10k_mean,sim,murcko,8310.2,9992.6,0.084239,0.831630,0.005433,0.000925,53.2
3,DrugEx_GT_epsilon_0.6_10k_mean,sim,murcko,8374.4,9949.8,0.189609,0.841634,0.007840,0.002239,76.6
4,DrugEx_RNN_epsilon_0.1_10k_mean,sim,murcko,6808.8,9780.0,0.036077,0.695723,0.020233,0.000616,183.2
5,DrugEx_RNN_epsilon_0.6_10k_mean,sim,murcko,7562.2,9874.0,0.140067,0.765754,0.011942,0.001731,113.8
6,GB_GA_mut_r_0.01_10k_mean,sim,murcko,4822.8,9128.6,0.031827,0.519768,0.003857,0.000623,34.6
7,GB_GA_mut_r_0.5_10k_mean,sim,murcko,3172.2,8637.6,0.068460,0.366520,0.037009,0.002174,289.8
8,addcarbon_10k_mean,sim,murcko,2513.6,10000.0,0.084125,0.251360,0.001965,0.002473,19.6


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_100k_mean,sim,murcko,35441.0,100085.0,0.199072,0.354114,0.003783,0.000512,374.8
1,REINVENT_100k_mean,sim,murcko,76475.0,98946.0,0.122156,0.772903,0.001213,0.000144,119.4
2,DrugEx_GT_epsilon_0.1_100k_mean,sim,murcko,71259.4,99908.6,0.279998,0.713227,0.005562,0.000356,545.8
3,DrugEx_GT_epsilon_0.6_100k_mean,sim,murcko,72822.6,99551.4,0.478395,0.731474,0.008490,0.000593,827.4
4,DrugEx_RNN_epsilon_0.1_100k_mean,sim,murcko,54286.0,97936.0,0.067967,0.553828,0.019543,0.000128,1775.0
5,DrugEx_RNN_epsilon_0.6_100k_mean,sim,murcko,62557.4,98693.2,0.345641,0.633724,0.012133,0.000523,1154.8
6,GB_GA_mut_r_0.01_100k_mean,sim,murcko,35417.0,91268.4,0.046377,0.380490,0.003486,0.000123,314.0
7,GB_GA_mut_r_0.5_100k_mean,sim,murcko,20788.2,86439.0,0.153584,0.239823,0.037504,0.000720,2931.6
8,addcarbon_100k_mean,sim,murcko,4977.4,100000.0,0.118272,0.049774,0.001720,0.001704,171.6


,name,type_cluster,scaffold,USo,SSo,TUPOR,SESY,ASER,ACR,CwASo
0,Molpher_500k_mean,sim,murcko,165200.2,500043.6,0.377033,0.330372,0.004411,0.000206,2180.8
1,REINVENT_500k_mean,sim,murcko,301652.2,494662.6,0.265722,0.609822,0.001221,0.000078,601.2
2,DrugEx_GT_epsilon_0.1_500k_mean,sim,murcko,305531.2,499569.0,0.394796,0.611569,0.005689,0.000118,2789.0
3,DrugEx_GT_epsilon_0.6_500k_mean,sim,murcko,321570.4,497777.2,0.608622,0.645980,0.008561,0.000171,4169.6
4,DrugEx_RNN_epsilon_0.1_500k_mean,sim,murcko,223237.8,489755.2,0.091614,0.455400,0.019674,0.000041,8934.6
5,DrugEx_RNN_epsilon_0.6_500k_mean,sim,murcko,265034.6,493474.2,0.495497,0.536941,0.011959,0.000173,5696.8
6,GB_GA_mut_r_0.01_500k_mean,sim,murcko,135277.4,456499.8,0.054980,0.290150,0.003488,0.000041,1575.2
7,GB_GA_mut_r_0.5_500k_mean,sim,murcko,77272.2,432117.4,0.243284,0.178290,0.037818,0.000294,14761.8
8,addcarbon_500k_mean,sim,murcko,5150.8,500000.0,0.118272,0.010302,0.001710,0.001652,853.0
